m# RETO RAG — Implementación End-to-End (Paso 3)

Este notebook implementa un **sistema RAG (Retrieval-Augmented Generation)** en entorno local (Anaconda + PyCharm).
Incluye: ingestión de PDF, chunking, embeddings, base vectorial **ChromaDB**, y pipeline de Recuperación + Generación.

**Requisitos mínimos**: `pip install --no-cache-dir langchain langchain-openai chromadb pypdf`
(Opcionales: `tiktoken`, `papermill`)

## 0) Configuración inicial

In [15]:
import os
from dotenv import load_dotenv

# Cargar variables desde .env
load_dotenv()

# Verificar que está cargada
print("API Key presente:", "OPENAI_API_KEY" in os.environ)

API Key presente: True


In [16]:

import os
from pathlib import Path

PDFS_DIR = Path("docs")
PDF_FILE = PDFS_DIR / "ejemplo.pdf"
CHROMA_PERSIST_DIR = Path("./.chroma_db_local")

OPENAI_EMBED_MODEL = "text-embedding-3-small"
OPENAI_CHAT_MODEL  = "gpt-4o-mini"

PDFS_DIR.mkdir(exist_ok=True, parents=True)
CHROMA_PERSIST_DIR.mkdir(exist_ok=True, parents=True)

print("OPENAI_API_KEY presente:", bool(os.getenv("OPENAI_API_KEY")))


OPENAI_API_KEY presente: True


## 1) Ingesta de PDF → Texto

In [5]:

from pypdf import PdfReader

def extract_text_from_pdf(pdf_path: Path) -> str:
    reader = PdfReader(str(pdf_path))
    return "\n".join([p.extract_text() or "" for p in reader.pages])

if PDF_FILE.exists():
    raw_text = extract_text_from_pdf(PDF_FILE)
    print("Longitud del texto extraído:", len(raw_text))
else:
    raw_text = ""
    print(f"Coloca un PDF en {PDF_FILE} y vuelve a ejecutar esta celda.")


Longitud del texto extraído: 73876


## 2) Chunking

In [7]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(text: str, chunk_size=500, chunk_overlap=80):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
        length_function=len,
    )
    return splitter.split_text(text)

chunks = chunk_text(raw_text) if raw_text else []
print(f"Total de fragmentos: {len(chunks)}")
if chunks[:1]:
    print("Ejemplo de fragmento:\n", chunks[0][:200], "...")


Total de fragmentos: 189
Ejemplo de fragmento:
 I n n o v a c i ó n  y  m e j o r a  C o n t i n u aG e s t i ó n  I n t e g r a l  d e  I n v e n t a r i o s   “ s y n c F l o w ”
G e s t i ó n  I n t e g r a l  D e  
I n v e n t a r i o s
S y n c ...


## 3) Embeddings + ChromaDB

In [10]:

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

def build_vector_store(text_chunks, persist_dir: Path):
    if not text_chunks:
        raise ValueError("No hay fragmentos para indexar. Verifica el PDF y el chunking.")
    emb = OpenAIEmbeddings(model=OPENAI_EMBED_MODEL)
    # Al crear con persist_directory, Chroma persiste automáticamente (>=0.4)
    db = Chroma.from_texts(text_chunks, emb, persist_directory=str(persist_dir))
    return db

db = None
if chunks:
    db = build_vector_store(chunks, CHROMA_PERSIST_DIR)
    print("ChromaDB inicializada en:", CHROMA_PERSIST_DIR)
else:
    print("No se construyó la base vectorial (sin fragmentos).")


ChromaDB inicializada en: .chroma_db_local


## 4) Recuperación + Generación

In [11]:

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

def build_rag_chain(chroma_db, chat_model=OPENAI_CHAT_MODEL, k=4):
    llm = ChatOpenAI(model=chat_model)
    retriever = chroma_db.as_retriever(search_kwargs={"k": k})
    qa = RetrievalQA.from_chain_type(
        llm=llm, retriever=retriever, return_source_documents=True
    )
    return qa

qa = build_rag_chain(db) if db else None
if qa: print("Pipeline RAG listo.")


Pipeline RAG listo.


## 5) Prueba rápida

In [12]:

query = "¿Cuáles son los conceptos clave mencionados en el documento?"
if qa and os.getenv("OPENAI_API_KEY"):
    result = qa.invoke({"query": query})
    print("Respuesta:", result["result"])
    print("\n---\nFragmentos usados:")
    for i, doc in enumerate(result["source_documents"], start=1):
        print(f"[{i}] {doc.page_content[:200]}...")
elif qa and not os.getenv("OPENAI_API_KEY"):
    print("Configura OPENAI_API_KEY antes de consultar.")


Respuesta: Los conceptos clave mencionados en el documento son:

1. Aplicar políticas detectadas
2. Descartar registros
3. Descartar productos con ciertos estados (Terminación, No Definido)
4. Descartar productos marcados como "avoid replenishment"
5. Reglas de administración de catálogo
6. Administrar catálogo
7. Categorización según propósito
8. Listado de productos
9. Pronósticos de ventas
10. Distribución de inventario
11. Cálculo de tamaños
12. Participación en catálogo
13. Distribución de catálogo

---
Fragmentos usados:
[1] A p l i c a r  l a s  p o l i t i c a s  d e t e c t a d a s
D e s c a r t a r  p o r  n o  c o n t a r  c o n  i n v e n t a r i o  i n i c i a l ,  e n t r a d a s ,  s a l i d a s  y  
v e n t a s
...
[2] A p l i c a r  l a s  p o l i t i c a s  d e t e c t a d a s
D e s c a r t a r  p o r  n o  c o n t a r  c o n  i n v e n t a r i o  i n i c i a l ,  e n t r a d a s ,  s a l i d a s  y  
v e n t a s
...
[3] ✓ R e g l a s  d e  a d m i n i s t r a c i ó n

In [13]:
query = "¿De que trata el concepto hsbt?"
if qa and os.getenv("OPENAI_API_KEY"):
    result = qa.invoke({"query": query})
    print("Respuesta:", result["result"])
    print("\n---\nFragmentos usados:")
    for i, doc in enumerate(result["source_documents"], start=1):
        print(f"[{i}] {doc.page_content[:200]}...")
elif qa and not os.getenv("OPENAI_API_KEY"):
    print("Configura OPENAI_API_KEY antes de consultar.")


Respuesta: No sé.

---
Fragmentos usados:
[1] R e g l a s  p a r a  e l  c á l c u l o  d e  H S B T
E s  l a  c l a s i f i c a c i ó n  q u e  s e  l e s  d a  a  l o s  p r o d u c t o s  d e  a c u e r d o  a  s u  p a r t i c i p a c i ó n  ...
[2] R e g l a s  p a r a  e l  c á l c u l o  d e  H S B T
E s  l a  c l a s i f i c a c i ó n  q u e  s e  l e s  d a  a  l o s  p r o d u c t o s  d e  a c u e r d o  a  s u  p a r t i c i p a c i ó n  ...
[3] R e g l a s  p a r a  e l  c á l c u l o  d e  H S B T
E s  l a  c a n t i d a d  d e  i n v e n t a r i o  c o n s u m i d o  e n  r e f e r e n c i a  a l  b u f f e r ,  e s  d e c i r ,  u n a  p ...
[4] R e g l a s  p a r a  e l  c á l c u l o  d e  H S B T
E s  l a  c a n t i d a d  d e  i n v e n t a r i o  c o n s u m i d o  e n  r e f e r e n c i a  a l  b u f f e r ,  e s  d e c i r ,  u n a  p ...


In [14]:
query = "¿Regla para el cálculo de Hsbt?"
if qa and os.getenv("OPENAI_API_KEY"):
    result = qa.invoke({"query": query})
    print("Respuesta:", result["result"])
    print("\n---\nFragmentos usados:")
    for i, doc in enumerate(result["source_documents"], start=1):
        print(f"[{i}] {doc.page_content[:200]}...")
elif qa and not os.getenv("OPENAI_API_KEY"):
    print("Configura OPENAI_API_KEY antes de consultar.")

Respuesta: La regla para el cálculo de Hsbt es una clasificación que se les da a los productos de acuerdo a su participación en la venta. Puede ser calculado a nivel catálogo, segmento o familia, según el lugar donde se consulta. Además, se menciona que la penetración indica la cantidad de inventario consumido en referencia al buffer, donde una penetración de 100% indica que todo el inventario se ha agotado y una penetración de 10% indica que tienes el 90% de inventario disponible.

---
Fragmentos usados:
[1] R e g l a s  p a r a  e l  c á l c u l o  d e  H S B T
E s  l a  c l a s i f i c a c i ó n  q u e  s e  l e s  d a  a  l o s  p r o d u c t o s  d e  a c u e r d o  a  s u  p a r t i c i p a c i ó n  ...
[2] R e g l a s  p a r a  e l  c á l c u l o  d e  H S B T
E s  l a  c l a s i f i c a c i ó n  q u e  s e  l e s  d a  a  l o s  p r o d u c t o s  d e  a c u e r d o  a  s u  p a r t i c i p a c i ó n  ...
[3] R e g l a s  p a r a  e l  c á l c u l o  d e  H S B T
E s  l a  c a 